# This notebook trains the Fine-Grained object classification on *main_logo* images. For more detailed explanation please see the notebook *FineGrain-zippepull*

In [ ]:
# This cell is not part of training process and only makes a text file that is later used in
# Dataset creation. It assumes that the images are splitted into two directories: 
# 1) fake 2) real. If they're in real directory the label is 0 and otherwise 1.
# It is not the only way to to that but it's one way around.

import cv2
import os
import glob

src_dir = '/content/drive/MyDrive/LV_data/main_zipper_pull/val/'

def parent_dir(item):
    return [item.rsplit('/')[-2]]


label_dict = {"real": 0, "fake": 1}

files = glob.glob(f'{src_dir}/*/*')

with open(src_dir + 'val.txt', 'w') as fid:
    for file in files:
        label = label_dict[parent_dir(file)[0]]
        fid.write('{0} {1}\n'.format(file, str(label)))
    fid.close()

In [ ]:
import torch
import shutil


def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, '/content/drive/MyDrive/'+filename)
    if is_best:
        shutil.copyfile('/content/drive/MyDrive/'+filename, '/content/drive/MyDrive/model_best.pth.tar')


class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count





def accuracy(scores, targets, k):
    """
    Computes top-k accuracy, from predicted and true labels.

    :param scores: scores from the model
    :param targets: true labels
    :param k: k in top-k accuracy
    :return: top-k accuracy
    """

    batch_size = targets.size(0)
    _, ind = scores.topk(k, 1, True, True)
    correct = ind.eq(targets.view(-1, 1).expand_as(ind))
    correct_total = correct.view(-1).float().sum()  # 0D tensor
    return correct_total.item() * (100.0 / batch_size)


In [ ]:
# Importing required packages.
from torch import nn

import torchvision
from torchvision import models
import torchvision.transforms as transforms

import torch.utils.data
from torch.utils.data import Dataset
from torch.utils.data.sampler import BatchSampler

import torch.nn.functional as F
import torch.optim

from PIL import Image
import numpy as np
import os
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.backends.cudnn as cudnn


In [ ]:
def default_loader(path):
    try:
        img = Image.open(path).convert('RGB')
    except:
        with open('read_error.txt', 'a') as fid:
            fid.write(path + '\n')
        return Image.new('RGB', (224, 224), 'white')
    return img


class RandomDataset(Dataset):
    def __init__(self, transform=None, dataloader=default_loader):
        self.transform = transform
        self.dataloader = dataloader

        with open('/content/drive/MyDrive/khaki2/val/val.txt', 'r') as fid:
            self.imglist = fid.readlines()

        self.labels = []
        for line in self.imglist:
            image_path, label = line.strip().split()
            self.labels.append(int(label))
        self.labels = np.array(self.labels)
        self.labels = torch.LongTensor(self.labels)

    def __getitem__(self, index):
        image_name, label = self.imglist[index].strip().split()
        image_path = image_name
        img = self.dataloader(image_path)
        img = self.transform(img)
        label = int(label)
        label = torch.LongTensor([label])

        return img, label

    def __len__(self):
        return len(self.imglist)


class BatchDataset(Dataset):
    def __init__(self, transform=None, dataloader=default_loader):
        self.transform = transform
        self.dataloader = dataloader

        with open('/content/drive/MyDrive/khaki2/train/train.txt', 'r') as fid:
            self.imglist = fid.readlines()

        self.labels = []
        for line in self.imglist:
            image_path, label = line.strip().split()
            self.labels.append(int(label))
        self.labels = np.array(self.labels)
        self.labels = torch.LongTensor(self.labels)

    def __getitem__(self, index):
        image_name, label = self.imglist[index].strip().split()
        image_path = image_name
        img = self.dataloader(image_path)
        img = self.transform(img)
        label = int(label)
        label = torch.LongTensor([label])

        return img, label

    def __len__(self):
        return len(self.imglist)


class BalancedBatchSampler(BatchSampler):
    def __init__(self, dataset, n_classes, n_samples):
        self.labels = dataset.labels
        self.labels_set = list(set(self.labels.numpy()))
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                 for label in self.labels_set}
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}
        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.dataset = dataset
        self.batch_size = self.n_samples * self.n_classes

    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < len(self.dataset):
            classes = np.random.choice(self.labels_set, self.n_classes, replace=False)
            indices = []
            for class_ in classes:
                indices.extend(self.label_to_indices[class_][
                               self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                         class_] + self.n_samples])
                self.used_label_indices_count[class_] += self.n_samples
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
            yield indices
            self.count += self.n_classes * self.n_samples

    def __len__(self):
        return len(self.dataset) // self.batch_size


In [ ]:

def pdist(vectors):
    distance_matrix = -2 * vectors.mm(torch.t(vectors)) + vectors.pow(2).sum(dim=1).view(1, -1) + vectors.pow(2).sum(
        dim=1).view(-1, 1)
    return distance_matrix

class API_Net(nn.Module):
    def __init__(self):
        super(API_Net, self).__init__()

        resnet101 = models.resnet101(pretrained=True)
        layers = list(resnet101.children())[:-2]

        self.conv = nn.Sequential(*layers)
        self.avg = nn.AvgPool2d(kernel_size=14, stride=1)
        self.map1 = nn.Linear(2048 * 2, 512)
        self.map2 = nn.Linear(512, 2048)
        self.fc = nn.Linear(2048, 2)
        self.drop = nn.Dropout(p=0.5)
        self.sigmoid = nn.Sigmoid()


    def forward(self, images, targets=None, flag='train'):
        conv_out = self.conv(images)
        pool_out = self.avg(conv_out).squeeze()

        if flag == 'train':
            intra_pairs, inter_pairs, \
                    intra_labels, inter_labels = self.get_pairs(pool_out, targets)

            features1 = torch.cat([pool_out[intra_pairs[:, 0]], pool_out[inter_pairs[:, 0]]], dim=0)
            features2 = torch.cat([pool_out[intra_pairs[:, 1]], pool_out[inter_pairs[:, 1]]], dim=0)
            labels1 = torch.cat([intra_labels[:, 0], inter_labels[:, 0]], dim=0)
            labels2 = torch.cat([intra_labels[:, 1], inter_labels[:, 1]], dim=0)


            mutual_features = torch.cat([features1, features2], dim=1)
            map1_out = self.map1(mutual_features)
            map2_out = self.drop(map1_out)
            map2_out = self.map2(map2_out)


            gate1 = torch.mul(map2_out, features1)
            gate1 = self.sigmoid(gate1)

            gate2 = torch.mul(map2_out, features2)
            gate2 = self.sigmoid(gate2)

            features1_self = torch.mul(gate1, features1) + features1
            features1_other = torch.mul(gate2, features1) + features1

            features2_self = torch.mul(gate2, features2) + features2
            features2_other = torch.mul(gate1, features2) + features2

            logit1_self = self.fc(self.drop(features1_self))
            logit1_other = self.fc(self.drop(features1_other))
            logit2_self = self.fc(self.drop(features2_self))
            logit2_other = self.fc(self.drop(features2_other))

            return logit1_self, logit1_other, logit2_self, logit2_other, labels1, labels2

        elif flag == 'val':
            return self.fc(pool_out)


    def get_pairs(self, embeddings, labels):
        distance_matrix = pdist(embeddings).detach().cpu().numpy()

        labels = labels.detach().cpu().numpy().reshape(-1,1)
        num = labels.shape[0]
        dia_inds = np.diag_indices(num)
        lb_eqs = (labels == labels.T)
        lb_eqs[dia_inds] = False
        dist_same = distance_matrix.copy()
        dist_same[lb_eqs == False] = np.inf
        intra_idxs = np.argmin(dist_same, axis=1)

        dist_diff = distance_matrix.copy()
        lb_eqs[dia_inds] = True
        dist_diff[lb_eqs == True] = np.inf
        inter_idxs = np.argmin(dist_diff, axis=1)

        intra_pairs = np.zeros([embeddings.shape[0], 2])
        inter_pairs  = np.zeros([embeddings.shape[0], 2])
        intra_labels = np.zeros([embeddings.shape[0], 2])
        inter_labels = np.zeros([embeddings.shape[0], 2])
        for i in range(embeddings.shape[0]):
            intra_labels[i, 0] = labels[i]
            intra_labels[i, 1] = labels[intra_idxs[i]]
            intra_pairs[i, 0] = i
            intra_pairs[i, 1] = intra_idxs[i]

            inter_labels[i, 0] = labels[i]
            inter_labels[i, 1] = labels[inter_idxs[i]]
            inter_pairs[i, 0] = i
            inter_pairs[i, 1] = inter_idxs[i]

        intra_labels = torch.from_numpy(intra_labels).long().to(device)
        intra_pairs = torch.from_numpy(intra_pairs).long().to(device)
        inter_labels = torch.from_numpy(inter_labels).long().to(device)
        inter_pairs = torch.from_numpy(inter_pairs).long().to(device)

        return intra_pairs, inter_pairs, intra_labels, inter_labels


In [ ]:
workers=2
epochs=80
start_epoch=0
lr=0.01
momentum=0.9
weight_decay=5e-4
print_freq=10
evaluate_freq=10
resume='/content/drive/MyDrive/checkpoint.pth.tar'
n_classes=2
n_samples=7

In [ ]:
def train(train_loader, model, criterion, optimizer_conv, scheduler_conv, optimizer_fc, scheduler_fc, epoch, step):
    best_prec1 = 0
    batch_time = AverageMeter()
    data_time = AverageMeter()
    softmax_losses = AverageMeter()
    rank_losses = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    
    # switch to train mode
    end = time.time()
    rank_criterion = nn.MarginRankingLoss(margin=0.05)
    softmax_layer = nn.Softmax(dim=0).to(device)

    for i, (input, target) in enumerate(train_loader):
        model.train()

        # measure data loading time
        data_time.update(time.time() - end)
        input_var = input.to(device)
        target_var = target.to(device).squeeze()

        # compute output
        logit1_self, logit1_other, logit2_self, logit2_other, labels1, labels2 = model(input_var, target_var,flag='train')
        batch_size = logit1_self.shape[0]
        labels1 = labels1.to(device)
        labels2 = labels2.to(device)

        self_logits = torch.zeros(2 * batch_size, 2).to(device)
        other_logits = torch.zeros(2 * batch_size, 2).to(device)
        self_logits[:batch_size] = logit1_self
        self_logits[batch_size:] = logit2_self
        other_logits[:batch_size] = logit1_other
        other_logits[batch_size:] = logit2_other

        # compute loss
        logits = torch.cat([self_logits, other_logits], dim=0)
        targets = torch.cat([labels1, labels2, labels1, labels2], dim=0)
    
        softmax_loss = criterion(logits, targets)

        self_scores = softmax_layer(self_logits)[torch.arange(2 * batch_size).to(device).long(),
                                                 torch.cat([labels1, labels2], dim=0)]
        other_scores = softmax_layer(other_logits)[torch.arange(2 * batch_size).to(device).long(),
                                                   torch.cat([labels1, labels2], dim=0)]
        flag = torch.ones([2 * batch_size, ]).to(device)
        rank_loss = rank_criterion(self_scores, other_scores, flag)

        loss = softmax_loss + rank_loss

        # measure accuracy and record loss
        prec1 = accuracy(logits, targets, 1)
        #prec2 = accuracy(logits, targets, 2)
        losses.update(loss.item(), 2 * batch_size)
        softmax_losses.update(softmax_loss.item(), 4 * batch_size)
        rank_losses.update(rank_loss.item(), 2 * batch_size)
        top1.update(prec1, 4 * batch_size)
        #top2.update(prec2, 4 * batch_size)

        # compute gradient and do SGD step
        optimizer_conv.zero_grad()
        optimizer_fc.zero_grad()
        loss.backward()
        if epoch >= 8:
            optimizer_conv.step()
        optimizer_fc.step()
        scheduler_conv.step()
        scheduler_fc.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Time: {time}\nStep: {step}\t Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'SoftmaxLoss {softmax_loss.val:.4f} ({softmax_loss.avg:.4f})\t'
                  'RankLoss {rank_loss.val:.4f} ({rank_loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                epoch, i, len(train_loader), batch_time=batch_time,
                data_time=data_time, loss=losses, softmax_loss=softmax_losses, rank_loss=rank_losses,
                top1=top1, step=step, time=time.asctime(time.localtime(time.time()))))

        if i == len(train_loader) - 1:
            val_dataset = RandomDataset(transform=transforms.Compose([
                transforms.Resize([512, 512]),
                transforms.CenterCrop([448, 448]),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)
                )]))
            val_loader = torch.utils.data.DataLoader(
                val_dataset, batch_size=batch_size, shuffle=False,
                num_workers=workers, pin_memory=True)
            prec1 = validate(val_loader, model, criterion)

            # remember best prec@1 and save checkpoint
            is_best = prec1 > best_prec1
            best_prec1 = max(prec1, best_prec1)
            save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'best_prec1': best_prec1,
                'optimizer_conv': optimizer_conv.state_dict(),
                'optimizer_fc': optimizer_fc.state_dict(),
            }, is_best)

        step = step + 1
    return step

In [ ]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter()
    softmax_losses = AverageMeter()
    top1 = AverageMeter()
    #top2 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    end = time.time()

    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):

            input_var = input.to(device)
            target_var = target.to(device).squeeze()

            # compute output
            logits = model(input_var, targets=None, flag='val')
            #print(logits)
            #print(target_var)
            softmax_loss = criterion(logits, target_var)
            

            prec1 = accuracy(logits, target_var, 1)
            #prec2 = accuracy(logits, target_var, 2)
            softmax_losses.update(softmax_loss.item(), logits.size(0))
            top1.update(prec1, logits.size(0))
            #top2.update(prec2, logits.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:
                print('Time: {time}\nTest: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'SoftmaxLoss {softmax_loss.val:.4f} ({softmax_loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                    i, len(val_loader), batch_time=batch_time, softmax_loss=softmax_losses,
                    top1=top1, time=time.asctime(time.localtime(time.time()))))
        print(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))

    return top1.avg


In [ ]:
torch.manual_seed(2)
torch.cuda.manual_seed_all(2)
np.random.seed(2)
epochs=250
# create model
model = API_Net()
model = model.to(device)
model.conv = nn.DataParallel(model.conv)

# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer_conv = torch.optim.SGD(model.conv.parameters(), lr,
                                  momentum=momentum,
                                  weight_decay=weight_decay)

fc_parameters = [value for name, value in model.named_parameters() if 'conv' not in name]
optimizer_fc = torch.optim.SGD(fc_parameters, lr,
                                momentum=momentum,
                                weight_decay=weight_decay)
if resume:
    if os.path.isfile(resume):
        print('loading checkpoint {}'.format(resume))
        checkpoint = torch.load(resume)
        start_epoch = checkpoint['epoch']
        best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer_conv.load_state_dict(checkpoint['optimizer_conv'])
        optimizer_fc.load_state_dict(checkpoint['optimizer_fc'])
        print('loaded checkpoint {}(epoch {})'.format(resume, checkpoint['epoch']))
    else:
        print('no checkpoint found at {}'.format(resume))

cudnn.benchmark = True
# Data loading code
train_dataset = BatchDataset(transform=transforms.Compose([
    transforms.Resize([512, 512]),
    transforms.RandomCrop([448, 448]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    )]))

train_sampler = BalancedBatchSampler(train_dataset, n_classes, n_samples)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_sampler, num_workers=workers, pin_memory=True)
scheduler_conv = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_conv, 100 * len(train_loader))
scheduler_fc = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_fc, 100 * len(train_loader))

step = 0
print('START TIME:', time.asctime(time.localtime(time.time())))
for epoch in range(start_epoch, epochs):
    step = train(train_loader, model, criterion, optimizer_conv, scheduler_conv, optimizer_fc, scheduler_fc, epoch, step)


loading checkpoint /content/drive/MyDrive/checkpoint.pth.tar
loaded checkpoint /content/drive/MyDrive/checkpoint.pth.tar(epoch 150)
START TIME: Tue Mar 29 21:06:56 2022
Time: Tue Mar 29 21:07:08 2022
Step: 0	 Epoch: [150][0/73]	Time 11.549 (11.549)	Data 9.048 (9.048)	Loss 0.0559 (0.0559)	SoftmaxLoss 0.0004 (0.0004)	RankLoss 0.0555 (0.0555)	Prec@1 100.000 (100.000)
Time: Tue Mar 29 21:07:27 2022
Step: 10	 Epoch: [150][10/73]	Time 3.792 (2.796)	Data 3.347 (2.161)	Loss 0.0547 (0.0600)	SoftmaxLoss 0.0047 (0.0080)	RankLoss 0.0500 (0.0520)	Prec@1 100.000 (100.000)
Time: Tue Mar 29 21:07:48 2022
Step: 20	 Epoch: [150][20/73]	Time 3.862 (2.477)	Data 3.417 (1.933)	Loss 0.0560 (0.0569)	SoftmaxLoss 0.0008 (0.0049)	RankLoss 0.0552 (0.0520)	Prec@1 100.000 (100.000)
Time: Tue Mar 29 21:08:10 2022
Step: 30	 Epoch: [150][30/73]	Time 3.887 (2.370)	Data 3.442 (1.856)	Loss 0.2705 (0.0631)	SoftmaxLoss 0.2168 (0.0113)	RankLoss 0.0537 (0.0517)	Prec@1 83.929 (99.482)
Time: Tue Mar 29 21:08:26 2022
Step: 40	 

# Cell below are for testing phase. See notebook *FineGrain-zipperpull* for details.

In [ ]:
class RandomDatasetTest(Dataset):
    def __init__(self, transform=None, dataloader=default_loader):
        self.transform = transform
        self.dataloader = dataloader

        with open('/content/drive/MyDrive/khaki2/test/test.txt', 'r') as fid:
            self.imglist = fid.readlines()

        self.labels = []
        for line in self.imglist:
            image_path, label = line.strip().split()
            self.labels.append(int(label))
        self.labels = np.array(self.labels)
        self.labels = torch.LongTensor(self.labels)

    def __getitem__(self, index):
        image_name, label = self.imglist[index].strip().split()
        image_path = image_name
        img = self.dataloader(image_path)
        img = self.transform(img)
        label = int(label)
        label = torch.LongTensor([label])

        return img, label

    def __len__(self):
        return len(self.imglist)


In [ ]:
model = API_Net()
model = model.to(device)
model.conv = nn.DataParallel(model.conv)
checkpoint = torch.load('/content/drive/MyDrive/model_best.pth.tar',map_location=torch.device('cpu'))

model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [ ]:
def confusion(scores, targets):
    """ Returns the confusion matrix for the values in the `prediction` and `truth`
    tensors, i.e. the amount of positions where the values of `prediction`
    and `truth` are
    - 1 and 1 (True Positive)
    - 1 and 0 (False Positive)
    - 0 and 0 (True Negative)
    - 0 and 1 (False Negative)
    """

    _, ind = scores.topk(1, 1, True, True)
    correct = ind.eq(targets.view(-1, 1).expand_as(ind))
    prediction = correct.view(-1).float()
    #correct_total = correct.view(-1).float().sum()  # 0D tensor
    confusion_vector = prediction / targets
    # Element-wise division of the 2 tensors returns a new tensor which holds a
    # unique value for each case:
    #   1     where prediction and truth are 1 (True Positive)
    #   inf   where prediction is 1 and truth is 0 (False Positive)
    #   nan   where prediction and truth are 0 (True Negative)
    #   0     where prediction is 0 and truth is 1 (False Negative)

    true_positives = torch.sum(confusion_vector == 1).item()
    false_positives = torch.sum(confusion_vector == float('inf')).item()
    true_negatives = torch.sum(torch.isnan(confusion_vector)).item()
    false_negatives = torch.sum(confusion_vector == 0).item()

    return true_positives, false_positives, true_negatives, false_negatives

In [ ]:
test_dataset = RandomDatasetTest(transform=transforms.Compose([
                transforms.Resize([512, 512]),
                transforms.CenterCrop([448, 448]),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)
                )]))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False,
                num_workers=workers, pin_memory=True)

criterion = nn.CrossEntropyLoss().to(device)

TP=0
TN=0
FP=0
FN=0
for input, target in test_loader:
  input_var = input.to(device)
  target_var = target.to(device).squeeze()

  logits = model(input_var, targets=None, flag='val')
  tp, fp, tn, fn = confusion(logits, target_var)

  TP += tp
  FP += fp
  TN += tn
  FN += fn
sp, se = TN / (TN+FP), TP / (TP+FN)
print('specificity: {}\t sensitivity: {}'.format(sp, se))

specificity: 0.45454545454545453	 sensitivity: 0.47058823529411764
